In [ ]:
# pip install satpy pyresample

In [ ]:
# !sudo apt-get update
# !sudo apt-get install python3.10

In [ ]:
# New Method

In [ ]:
# !pip install ee
# !pip install geemap

In [ ]:
import ee
import geemap

In [ ]:
!sudo -i earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bFy2kQBKdiWHl_Lw5SY1VRPZrQC43ZPJwQ-lMHbCijU&tc=aXHmKe9oWTbKFsutmzipM8tyv9axfL3Gl3VhE8W51LY&cc=XXHkYInFuhmB4sgPuVB1MSwJMarwg3BYhoqqNQ_qiLA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 

In [ ]:
ee.Initialize()

In [ ]:
map1 = geemap.Map()
map1

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
map1.setCenter(86,8,9)

In [ ]:
# map1 = ee.Geometry.Rectangle([], None, False)

In [ ]:
map1

In [ ]:
point = ee.Geometry.Point(86,8);

In [ ]:
# data = ee.ImageCollection("COPERNICUS/S3/OLCI").filterBounds(point)
data = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(point)

In [ ]:
# image = ee.Image(data.filterDate("2023-08-15","2023-09-15").sort("saturatedPixelsPercent").first())
# image = ee.Image(data.filterDate("2023-09-15","2023-09-16").sort("brightPixelsPercent").first())
# image = ee.Image(data.filterDate("2023-09-15","2023-09-23"))

In [ ]:
image

In [ ]:
# ndvi = image.expression(
#     "(b1 - b2)/ (b1 + b2)",
#     {"b1":image.select("Oa04_radiance"), "b2": image.select("Oa05_radiance")})

In [ ]:
ndvi = image.expression(
    "(b1-b2)/(b1+b2)",
    {"b1":image.select("Oa05_radiance"), "b2": image.select("Oa05_radiance")})

In [ ]:
display = {
    "min":0,
    "max":1,
    "palette":['Black',"Green"]
}

In [ ]:
map1.addLayer(ndvi,display)
map1

#new

In [ ]:
map1 = geemap.Map()

In [ ]:
# point = ee.Geometry.Point(135,40);
point = ee.Geometry.Point([87,])

xMin = 86
yMin = 8
xMax = 90
yMax = 8

# point = ee.Geometry.Rectangle([xMin, yMin, xMax, yMax]);

map1.centerObject(point);

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterBounds(point).filterDate('2023-09-02', '2023-09-03');

# Colour
rgb = data.select(['Oa03_radiance', 'Oa05_radiance', 'Oa08_radiance']).median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]));



In [ ]:
visParams = {
  min: 0,
  max: 6
}

In [ ]:
map1.addLayer(rgb, visParams, 'RGB');
map1

Map(center=[8.002406583093055, 88.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [ ]:
# New code

#Seninel 2

In [ ]:
#########################
# sentinel 2



In [ ]:
def mask_s2_clouds(image):
  qa = image.select('B12').subtract(image.select('B11'))

  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [ ]:
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2023-08-01', '2023-08-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('WATER_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)



In [ ]:
visualization = {
    'min': 0.0,
    'max': 1,
    'bands': ['B11','B12'],
}



In [ ]:
m = geemap.Map()
m.set_center(86, 8, 9)
m.add_layer(dataset.mean(), visualization, 'RG')
m